In [4]:
from os import link
from time import sleep
from helium import *
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, WebDriverException, ElementNotVisibleException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd


In [58]:
import fasttext
with open("cooking.stackexchange.txt",'r') as file:
    lines = file.readlines()

with open("test_orig",'w') as file:
    for line in lines[:int(len(lines)/3)]:
        file.write(line)
with open("train_orig",'w') as file:
    for line in lines[int(len(lines)/3):]:
        file.write(line)


In [63]:
model_orig = fasttext.train_supervised('train_orig')

In [64]:
results_orig = model_orig.test("./test_orig.txt")
print_results(*results_orig)

N	5133
P@1	0.090
R@1	0.039


In [66]:
model_orig = fasttext.train_supervised('train_orig', lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='ova')
results_orig = model_orig.test("./test_orig.txt")
print_results(*results_orig)

N	5133
P@1	0.513
R@1	0.224


In [4]:
bs = BeautifulSoup()
driver = start_chrome(headless=True)
driver.get('https://amazon.com.au')
click(TextField(to_right_of='All'))
write('RTX 3090')
click(Button(to_left_of='Hello, Sign In'))

all_links = []

links = driver.find_elements_by_xpath("//a[@href]")
for l in links:
    if 'Graphics' in l.get_attribute('href'):
        all_links.append(l.get_attribute('href'))

all_reviews = []

In [29]:
def scrap_link(link):
    driver = start_chrome(headless=True)
    driver.get(link)
    click('See all reviews')
    sleep(3)
    reviews = driver.find_elements_by_css_selector('#cm_cr-review_list div[data-hook="review"]')

    for i, result in enumerate(all_reviews):
        #data['reviews'].append(scrape_single_review(result))
        for r in result:
            data['reviews'].append(scrape_single_review(r))
            all_reviews.append(reviews)
            sleep(3)
    if(Button('Next page').exists()):
        click('Next page')
    sleep(5)

In [23]:
def scrape_single_review(result):
        selectors = {
            'body': '[data-hook="review-body"]',
        }
        results = dict()
        driver = start_chrome(headless=True)
        for key, selector in selectors.items():
            try:
                sleep(3)
                element = result.find_element_by_css_selector(selector)
                results[key] = element.text
                print('REVIEW:{}'.format(element.text))
            except NoSuchElementException as e:
                print('NoSuchElementException')

        try:
            results['author_url'] = result.find_element_by_css_selector('a[data-hook="review-author"]').get_attribute('href')
            results['rating'] = result.find_element_b=y_css_selector('i[data-hook="review-star-rating"] span').text
        except NoSuchElementException as e:
            print('Cannot scrap aditional data')


        return results

In [ ]:

data = {
            'reviews': [],
        }

for i, result in enumerate(all_reviews):
    #data['reviews'].append(scrape_single_review(result))
    for r in result:
        print(r.text)
        #data['reviews'].append(scrape_single_review(r))

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import datetime
import multiprocessing
import pandas as pd
import fasttext
import numpy as np

In [8]:
def get_sentiment(x):
    if x > 3:
        return 'Positive'
    elif x < 3:
        return 'Negative'
    else:
        return 'Neutral'

def clean_text(x):
    return re.sub(r"(@[A-Za-z0–9_]+)|[^\w\s]|#|http\S+", "", x)

def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

def get_X_y(df):
    return df['reviews.text'], df['sentiment']


In [9]:
gift_cards = pd.read_json('gift_cards.json', lines=True)


In [10]:
gift_cards

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,1,25,False,"12 19, 2008",APV13CM0919JD,B001GXRQW0,{'Gift Amount:': ' 50'},LEH,"Amazon,\nI am shopping for Amazon.com gift car...",Merry Christmas.,1229644800,NaN
1,5,NaN,False,"12 17, 2008",A3G8U1G1V082SN,B001GXRQW0,{'Gift Amount:': ' 50'},Tali,"I got this gift card from a friend, and it was...",Gift card with best selection,1229472000,NaN
2,5,4,False,"12 17, 2008",A11T2Q0EVTUWP,B001GXRQW0,{'Gift Amount:': ' 50'},Z,aren't we going to save trees?! :) People who ...,A convenient and great gift for the environmen...,1229472000,NaN
3,5,NaN,False,"12 17, 2008",A9YKGBH3SV22C,B001GXRQW0,{'Gift Amount:': ' 25'},Giotravels,You can always get someone something from Amaz...,Totally make sense,1229472000,NaN
4,1,NaN,True,"12 17, 2008",A34WZIHVF3OKOL,B001GXRQW0,NaN,King Dad,Why take 50 dollars of good money with no limi...,Give CASH!,1229472000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
147189,5,NaN,True,"09 15, 2018",A2K9WVQW9TLWNK,B01H5PPJT4,NaN,Mark.,I always enjoy getting these Gift cards via em...,Just always great service!!,1536969600,NaN
147190,4,NaN,True,"09 7, 2018",A149ALSR6TPGF7,B01H5PPJT4,NaN,timothy kuta,Worked great,Good to have,1536278400,NaN
147191,5,NaN,True,"08 29, 2018",A2Q066NZCQSCOR,B01H5PPJT4,NaN,Jess,Gift card,Gift card,1535500800,NaN
147192,5,NaN,True,"08 18, 2018",A1KJLWCW7XBS8I,B01H5PPJT4,NaN,Average Homeowner,"What is there to say, It's a gift card.",Easy to buy and give as a gift.,1534550400,NaN


In [11]:
gift_cards = gift_cards[['overall', 'reviewText']]
gift_cards.head()

,overall,reviewText
0,1,"Amazon,\nI am shopping for Amazon.com gift car..."
1,5,"I got this gift card from a friend, and it was..."
2,5,aren't we going to save trees?! :) People who ...
3,5,You can always get someone something from Amaz...
4,1,Why take 50 dollars of good money with no limi...


In [12]:
gift_cards.dropna(axis=0, inplace=True)


C:\Users\Dani\Anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
gift_cards['reviewText'] = gift_cards['reviewText'].apply(lambda x:clean_text(x))
gift_cards['sentiment'] = gift_cards['overall'].apply(lambda x:get_sentiment(x))

C:\Users\Dani\Anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Dani\Anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
X = gift_cards['reviewText']
y = gift_cards['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,test_size=0.2)

In [17]:
X_test

100647    simple to purchase and send to the receiver gr...
106300    Great way to order gift cards for Nieces who l...
13319     Gift card was easy to give and appreciated by ...
21406                                       Its a gift card
86538     Easy delivery nicely themed gift card recipien...
                                ...                        
27972                               Quick  Easy gift giving
58647     Always popular with my grandchildren My grandd...
5352      Arrived in time and its a gift cardso hopefull...
66589     So convinience for use every time stop by Star...
63832     I bought this for my mom birthday She really l...
Name: reviewText, Length: 29407, dtype: object

In [22]:
with open("./test", "w") as f_test:
    for X_test_f, y_test_f in zip(X_test, y_test):
        line = "__label__" + str(y_test_f) + "\t" + str(X_test_f) + "\n"
        try:
            f_test.write(line)
        except:
            print(line)
            break

In [23]:
with open("./train", "w") as f_train:
    for X_train_f, y_train_f in zip(X_train, y_train):
        line = "__label__" + str(y_train_f) + "\t" + str(X_train_f) + "\n"
        try:
            f_train.write(line)
        except:
            print(line)
            break

In [24]:
model = fasttext.train_supervised(input="./train.txt")

In [25]:
model.save_model('gift_model.bin')

In [27]:
results = model.test("./test")
print_results(*results)

N	29407
P@1	0.950
R@1	0.950


In [31]:
model = fasttext.train_supervised(input="./train", lr=0.1, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='ova')
results = model.test("./test")
print_results(*results)

N	29407
P@1	0.951
R@1	0.951
